<a href="https://colab.research.google.com/github/Mohint/Generative-AI/blob/main/Unsupervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn nltk gradio
!python -m nltk.downloader stopwords
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
import re
import pickle

# Download NLTK stopwords if not already downloaded
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

# --- 1. Load and Preprocess Data (for training) ---
# This part is typically done once to train and save the model
# For a deployed Gradio app, you'd load the pre-trained model and vectorizer

def preprocess_text(text):
    text = text.lower() # Lowercase
    text = re.sub(r'[^a-z\s]', '', text) # Remove punctuation and numbers
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')] # Remove stopwords
    return ' '.join(words)

# Load the dataset
try:
    df = pd.read_csv('spam.csv', encoding='latin-1')
    df = df.iloc[:, :2] # Keep only 'v1' (label) and 'v2' (text)
    df.columns = ['label', 'message']
except FileNotFoundError:
    print("spam.csv not found. Please download it from Kaggle and place it in the same directory.")
    print("URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset")
    exit()

# Preprocess messages
df['message'] = df['message'].apply(preprocess_text)

# Convert labels to numerical (ham: 0, spam: 1)
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Split data
X = df['message']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes Classifier
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Evaluate the model (optional, for verification)
y_pred = model.predict(X_test_vectorized)
print(f"Model accuracy: {accuracy_score(y_test, y_pred):.2f}")

# Save the trained model and vectorizer
with open('spam_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved successfully!")

# --- 2. Load the trained model and vectorizer (for Gradio app) ---
# This part is what runs when your Gradio app starts
try:
    with open('spam_model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        loaded_vectorizer = pickle.load(f)
    print("Loaded model and vectorizer for inference.")
except FileNotFoundError:
    print("Error: spam_model.pkl or tfidf_vectorizer.pkl not found.")
    print("Please ensure the model training section above has run successfully to create these files.")
    exit()

def predict_spam(message):
    """
    Predicts whether a message is spam or not.
    """
    if not message:
        return "Please enter a message."

    preprocessed_message = preprocess_text(message)
    vectorized_message = loaded_vectorizer.transform([preprocessed_message])
    prediction = loaded_model.predict(vectorized_message)[0]

    if prediction == 1:
        return "SPAM! 🚨"
    else:
        return "Not Spam (HAM) ✅"

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(lines=5, label="Enter Message Here", placeholder="Type a message to check if it's spam..."),
    outputs=gr.Label(label="Prediction"),
    title="SMS Spam Detector",
    description="Enter an SMS message to determine if it's spam or not. This model is trained on the SMS Spam Collection Dataset."
)

# Launch the Gradio app (use share=True for Google Colab)
if __name__ == "__main__":
    iface.launch(share=True) # For Colab, use share=True. For local, you can omit or set to False.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
spam.csv not found. Please download it from Kaggle and place it in the same directory.
URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset


NameError: name 'df' is not defined

In [ ]:
!pip install gradio
import gradio as gr
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import numpy as np

# Load dataset
iris = load_iris()
X = iris.data

# Fit KMeans model
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
kmeans.fit(X)

# Define prediction function
def predict_cluster(sepal_length, sepal_width, petal_length, petal_width):
    input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    cluster = kmeans.predict(input_data)[0]
    return f"Predicted Cluster: {cluster}"

# Gradio interface
iface = gr.Interface(
    fn=predict_cluster,
    inputs=[
        gr.Number(label="Sepal Length (cm)"),
        gr.Number(label="Sepal Width (cm)"),
        gr.Number(label="Petal Length (cm)"),
        gr.Number(label="Petal Width (cm)")
    ],
    outputs="text",
    title="Iris Clustering with KMeans (Unsupervised Learning)",
    description="This model uses KMeans clustering to group Iris flowers into clusters based on their features."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://58398fd01e4de3b673.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
